In [58]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
from tqdm import tqdm

In [49]:
PATH_URLS = "datahuck_sample.csv"

In [50]:
urls_df = pd.read_csv(PATH_URLS, index_col="seed_url")

- Chose to downgrade all urls to http.
- Will mark whether url has link to certain external websites and will count the total number of links to external websites

In [78]:
from urllib.parse import urljoin

def fetch_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.text
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

def extract_links(url):
    # Send a GET request to fetch the HTML content
    response = fetch_page(url)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response, 'html.parser')
    
    # Find all <a> tags with an href attribute
    links = soup.find_all('a', href=True)
    
    # Extract the href attribute from each <a> tag and store it in a list
    hrefs = [link['href'] for link in links]
    hrefs = [urljoin(url, link['href']) for link in links]
    hrefs = [url.replace(urlparse(url).scheme + ':', 'http:') for url in hrefs]

    return hrefs

def process_links(url, links):
    internal_links = [l for l in links if url in l]
    external_links = [l for l in links if url not in l]

    first_degree_children = set()
    params = []

    # print(internal_links)
    for int_link in internal_links:
        parsed_url = urlparse(int_link)
        if len(parsed_url.params) > 0:
            params.append({"PATH": parsed_url.path, "params": dict([param.split("=") for param in urlparse(url).query.split("&")])})

        c = parsed_url.path.strip("/").split("/")[0]
        if len(c) > 0:
            first_degree_children.add(c)
    
    # print(external_links)
    out_links = set()
    for ext_link in external_links:
        parsed_url = urlparse(ext_link)
        if parsed_url.hostname:
            out_links.add(parsed_url.hostname.lower().replace("www.", ""))

    # out_links = {urlparse(ext_link).hostname.lower().replace("www.", "") for ext_link in external_links if urlparse(url).hostname}
    return {
        "url": url,
        "children": first_degree_children,
        "params": params,
        "external_sites": out_links,
        "error?": False
    }
    # print(first_degree_children, params, out_links)
    # print("---------\n\n")

def process_url(url):
    try:
        return process_links(url=url, links=extract_links(url))
    except Exception as e:
        return {
        "url": url,
        "children": "",
        "params": "",
        "external_sites": "",
        "error?": e
    }


In [79]:
import numpy as np
segments = urls_p1, urls_p2, urls_p3, urls_p4 = np.array_split(urls_df, 4)

/Users/majdbishara/VisualStudio/Paypal_1/.conda/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [80]:
%%time
def batch_process(df, alias, batch_size=10):
    num_batches = len(df) // batch_size + (len(df) % batch_size > 0)
    processed_data = []
    print(f'Processing {alias} with batch size: {batch_size} and number of batches: {num_batches}')

    for i in tqdm(range(num_batches)):
        batch = urls_df.iloc[i * batch_size:(i + 1) * batch_size]
        for seed_url in batch.index:
            result = process_url(seed_url)
            processed_data.append(result)
        
        # Save intermediate results
        intermediate_df = pd.DataFrame(processed_data)
        intermediate_df.to_csv(f'intermediate_results_{alias}.csv', index=False)

    # Save final results
    final_df = pd.DataFrame(processed_data)
    final_df.to_csv(f'final_results_{alias}.csv', index=False)

for i, s in enumerate(segments):
    alias = f"parition_{i+1}"
    batch_process(s, alias)

Processing parition_1 with batch size: 10 and number of batches: 2500


  0%|          | 1/2500 [00:28<19:53:35, 28.66s/it]

Request failed: 406 Client Error: Not Acceptable for url: http://goslg.com/


  0%|          | 2/2500 [00:51<17:24:24, 25.09s/it]

Request failed: 406 Client Error: Not Acceptable for url: http://tedxrexburg.com/
Request failed: 404 Client Error: Not Found for url: http://wiserpiercing.com/
Request failed: 404 Client Error: Not Found for url: http://poopieandpooper.com/


...

Request failed: 406 Client Error: Not Acceptable for url: http://japantopcars.com/


 20%|██        | 512/2500 [4:12:40<17:27:21, 31.61s/it]

Request failed: 406 Client Error: Not Acceptable for url: http://sattiestrinipeppersauce.com/


 20%|██        | 512/2500 [4:12:46<16:21:28, 29.62s/it]


KeyboardInterrupt: 